In [1]:
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [14]:
env = gym.make('CartPole-v0')

In [15]:
env.observation_space

Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)

In [5]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.action_head = nn.Linear(128, 2)
        self.value_head = nn.Linear(128, 1)
        self.saved_actions = []
        self.rewards = []
    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_prob = F.softmax(self.action_head(x), dim=-1)
        state_values = self.value_head(x)
        return action_prob, state_values

In [6]:
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()

In [7]:
def finish_episode():
    """
    Training code. Calcultes actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = [] # list to save the true values

    for r in model.rewards[::-1]:
        R = r + 0.99 * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)

    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()

        policy_losses.append(-log_prob * advantage)

        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    optimizer.zero_grad()

    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    loss.backward()
    optimizer.step()

    del model.rewards[:]
    del model.saved_actions[:]

In [8]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

In [9]:
def train():
    running_reward = 10
    for i_episode in range(500):
        state = env.reset()
        ep_reward = 0
        for t in range(1, 10000):
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
        finish_episode()
        if i_episode % 10 == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

In [10]:
%%time
train()

Episode 0	Last reward: 18.00	Average reward: 10.40
Episode 10	Last reward: 53.00	Average reward: 23.09
Episode 20	Last reward: 121.00	Average reward: 45.18
Episode 30	Last reward: 200.00	Average reward: 65.18
Episode 40	Last reward: 11.00	Average reward: 69.70
Episode 50	Last reward: 8.00	Average reward: 45.56
Episode 60	Last reward: 10.00	Average reward: 31.20
Episode 70	Last reward: 9.00	Average reward: 22.42
Episode 80	Last reward: 9.00	Average reward: 17.10
Episode 90	Last reward: 10.00	Average reward: 14.17
Episode 100	Last reward: 9.00	Average reward: 12.25
Episode 110	Last reward: 10.00	Average reward: 11.17
Episode 120	Last reward: 10.00	Average reward: 10.56
Episode 130	Last reward: 10.00	Average reward: 10.06
Episode 140	Last reward: 10.00	Average reward: 9.85
Episode 150	Last reward: 9.00	Average reward: 9.86
Episode 160	Last reward: 9.00	Average reward: 9.50
Episode 170	Last reward: 9.00	Average reward: 9.57
Episode 180	Last reward: 10.00	Average reward: 9.70
Episode 190	La

In [11]:
from gym import wrappers

In [19]:
done = False
cnt = 0
# env = wrappers.Monitor(env, 'actor-critic-cartpole', force=True)

In [20]:
observation = env.reset()
while not done:
    cnt += 1
    action = select_action(observation)
    observation, reward, done, _ = env.step(action)
    if done:
        break

In [21]:
print('game lasted {} moves'.format(cnt))

game lasted 133 moves


In [22]:
torch.save(model.state_dict(), "actor_critic_cartpole.pth")